# 2025 09 25 - OpenAI API 

## 삼성전자 주가분석

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
default_model = os.getenv("OPENAI_DEFAULT_MODEL") 

client= OpenAI()

In [2]:
#1. ticker, 30 - 터커 회사, 현재를 기준으로 - 30일간 (8월26~9월25일)
#2. 해당회사의 30일간의 주식 내용을 가져오기 - get_stock_price(ticker, days)
#    df = 가져온 데이터(30일간의 데이터) return
#3. 리턴 받은 데이터 (df)를 분석하기 위해서 LLM을 호출(prompt) -> analyze_stock(df,company,days)
#   OHLCV -> df (open, high, low, close, volume값을 구하기) -> prompt -> LLM ->응답 -> return


In [3]:
from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta

def get_stock_price(ticker, days=30):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    #O(Open)H(High)L(Low)C(Close)V(Volume)
    df = stock.get_market_ohlcv(start_date.strftime("%Y%m%d"),
                                end_date.strftime("%Y%m%d"), 
                                ticker)
    return df                            

c:\Users\edukd\miniconda3\envs\myKDA2\Lib\site-packages\pykrx\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
df.head(4)

NameError: name 'df' is not defined

In [ ]:
def analyze_stock(df, company, days=30):
    if df.empty:
        return "분석할 데이터가 없습니다"

    latest_price = df['종가'].iloc[-1]
    highest_price = df['종가'].max()
    lowest_price = df['종가'].min()
    average_price = df['종가'].mean()
    
    analysis = (
        f"최근 {days}일 동안 {company}의 종가 데이터를 분석한 결과는 다음과 같습니다\n"
        f"Latest Price: {latest_price}\n"
        f"Highest Price in the period: {highest_price}\n"
        f"Lowest Price in the period: {lowest_price}\n"
        f"Average Price in the period: {average_price}"
        f"이 데이터를 기반으로 간단한 분석과 전망을 제공해주세요"
    )

    ret = client.chat.completions.create(
        model=default_model,
        messages=[{"role":"system", "content":"너는 주식분석 전문가야"},
                {"role":"user", "content":analysis}],
        temperature=0,
        max_tokens=500
    )
    return ret.choices[0].message.content.strip()

In [ ]:
if __name__ == "__main__":
    ticker = "005930"  # 삼성전자
    df = get_stock_price(ticker, days=30)
    ret = analyze_stock(df, '삼성전자', 30)    
    print(ret)

## 캠핑장 키워드 찾기

In [ ]:
camping_api = os.getenv("GO_CAMPING_API_KEY")
print(camping_api)

2xkTJclMbulkscqwYrQTe3UqgqAanShFY2F%2FdjvjhGBLU9pKh1DSlEjObRX2lJ47AZcyUcwfyILXveNbsZLTDA%3D%3D


In [ ]:
#키워드 추출
question = "계곡이 있는 캠핑장을 추천해줘"
question = "가족이 주말에 함께 여행할 캠핑장 추천해줘"
response = client.chat.completions.create(
    model=default_model,
    messages=[
        {"role": "system", "content": "다음 질문에서 가장 중요한 키워드 단어 하나만 뽑아줘"},
        {"role": "user", "content": question}
    ],
    temperature=0,
    max_tokens=256
)
keyword = response.choices[0].message.content.strip()
print(keyword)


캠핑장


In [ ]:

url=f'http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey={camping_api}&MobileOS=ETC&MobileApp=AppTest&keyword={keyword}&_type=json'
print(url)

In [ ]:
import requests

res = requests.get(url)
if res.status_code == 200:
    data = res.json()
else:
    print(f"Error : {res.status_code}")

result= data['response']['body']['items']['item']

context = ""
for item in result:
    context += item['facltNm']+":"+item['induty']+':'+item['doNm']+':'+item['intro']+'\n'

print(context)

In [ ]:
response = client.chat.completions.create(
    model=default_model,
    messages=[
        {"role": "system", "content": f"주어진 문단을 보고 다음 질문에 답해줘:{question}"},
        {"role": "user", "content": context}
    ],
    temperature=0,
    max_tokens=256
)
keyword = response.choices[0].message.content.strip()
print(keyword)

가족이 주말에 함께 여행할 캠핑장으로 다음을 추천합니다:

1. **파인트리캠핑장 (강원도 정선)**: 맑은 개울이 흐르고 물놀이와 민물낚시를 즐길 수 있는 곳으로, 아이들과 함께하기 좋은 수영장과 트램펄린이 있어 가족 단위 방문에 적합합니다.

2. **꿈꾸는 캠핑장 (전라남도 곡성)**: 다양한 나무들로 둘러싸인 숲 속에 위치하며, 할로윈 시즌에는 이색적인 이벤트가 있어 아이들이 즐길 수 있는 특별한 경험을 제공합니다.

3. **꿈꾸는 농부 오토캠핑장 (대전광역시)**: 청량감 가득한 환경에서 물놀이를 즐길 수 있으며, 인근에 유명 관광지가 많아 연계 관광이 용이합니다. 반려동물도 소형견에 한해 출입이 가능해 가족과 함께하는 캠핑에 좋습니다.

4. **꿈꾸는바다캠핑장 (충청남도 태안)**: 태안 앞바다를 바라보며


## functions / tools

In [ ]:
city = '서울'
response = client.chat.completions.create(
    model=default_model,
    messages=[{"role":"user", "content":f"{city}의 오늘 날씨 알려줘"}]
)
print(response.choices[0].message.content.strip())    

죄송하지만, 실시간 날씨 정보를 제공할 수는 없습니다. 서울의 오늘 날씨를 확인하시려면 기상청 웹사이트나 모바일 앱, 혹은 날씨 관련 웹사이트를 참조하시기 바랍니다. 도움이 필요하시면 다른 질문 해주세요!


In [ ]:
print(response)

ChatCompletion(id='chatcmpl-CJYiejGuGKXABVQQOrZdymsgIYWKU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='죄송하지만, 실시간 날씨 정보를 제공할 수는 없습니다. 서울의 오늘 날씨를 확인하시려면 기상청 웹사이트나 모바일 앱, 혹은 날씨 관련 웹사이트를 참조하시기 바랍니다. 도움이 필요하시면 다른 질문 해주세요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1758777728, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=57, prompt_tokens=14, total_tokens=71, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
def get_weather(city):
    weather_data = {
        "서울":"맑음, 25도",
        "부산":"비, 22도",
        "대구":"흐림, 24도",
        "인천":"맑음, 23도"
    }
    return weather_data.get(city,"정보없음")

def get_time(city):
    import datetime
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"{city}의 현재 시간은 {current_time}입니다"    

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "오늘의 날씨를 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "도시이름",
                    },
                },
                "required": ["city"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "현재 시간을 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "time zone",
                    },
                },
                "required": ["city"],
            }
        }
    }
]

In [ ]:
city = '서울'
response = client.chat.completions.create(
    model=default_model,
    messages=[{"role":"user", "content":f"{city}의 현재 시간 알려줘"}],
    tools=tools,
    tool_choice="auto"
)
print(response.choices[0].message.content)  

None


In [ ]:
print(response.choices[0].message.tool_calls[0].function.name)

get_time


In [ ]:
#사용가능한 함수들을 딕셔너리로 매핑
available_functions = {
    "get_weather": get_weather,
    "get_time": get_time
}

import json
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    print(f'{tool_call.function.name}')
    print(f'{tool_call.function.arguments}')

    # 동적으로 함수 호출
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    if function_name in available_functions:
        function_to_call = available_functions[function_name]
        weather_info = function_to_call(**function_args)
        print(weather_info)
    else:
        print(f"함수 {function_name}을 찾을 수 없습니다.")

get_time
{"city":"Asia/Seoul"}
Asia/Seoul의 현재 시간은 2025-09-25 14:39:47입니다
